In [134]:
import numpy as np
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px

from Project.Database import Db
from Project._05InferKnowledgeOfRules.infer_rules_functions import json_to_dataframe

# Phase 1
Define available appliances

In [135]:
meta = Db.load_data(meta=True, hourly=False, consumption=False)
appliance_list = meta.loc[(lambda self:
                           (self["Parameter"] == "Status_OnOff") &
                           (~ self.index.str.contains("SensHeat")) &
                           (~ self.index.str.contains("Cold")))].index.to_list()
appliance_list

['DHW_StatusSolenoidHotKitchenSink',
 'DHW_StatusSolenoidHotMBAShower',
 'DHW_StatusSolenoidHotMBATub',
 'Load_StatusApplianceCooktop',
 'Load_StatusApplianceDishwasher',
 'Load_StatusApplianceOven',
 'Load_StatusApplianceRangeHood',
 'Load_StatusBA1Lights',
 'Load_StatusBA2Lights',
 'Load_StatusBR2Lights',
 'Load_StatusBR3Lights',
 'Load_StatusBR4Lights',
 'Load_StatusDRLights',
 'Load_StatusEntryHallLights',
 'Load_StatusKitchenLightsA',
 'Load_StatusKitchenLightsB',
 'Load_StatusKitchenLightsC',
 'Load_StatusLRLights1',
 'Load_StatusLRLights2',
 'Load_StatusLRLights3',
 'Load_StatusLatentload',
 'Load_StatusMBALights',
 'Load_StatusMBRLights1',
 'Load_StatusMBRLights2',
 'Load_StatusPlugLoadBR2Laptop',
 'Load_StatusPlugLoadBR3Laptop',
 'Load_StatusPlugLoadBlender',
 'Load_StatusPlugLoadCanOpener',
 'Load_StatusPlugLoadCoffeeMaker',
 'Load_StatusPlugLoadDesktopPCMonitor',
 'Load_StatusPlugLoadFan',
 'Load_StatusPlugLoadHairDryerCurlIron',
 'Load_StatusPlugLoadHandMixer',
 'Load_Statu

In [136]:
meta[['Load_Match', 'Elec_Match', 'DHW_Match']]

,Load_Match,Elec_Match,DHW_Match
Unnamed: 0,,,
DHW_ClothesWasherColdFlow,NaN,NaN,NaN
DHW_ClothesWasherHotFlow,NaN,NaN,NaN
DHW_DishwasherHotFlow,NaN,NaN,NaN
DHW_HeatPumpWaterHeaterEnergyTotal,NaN,NaN,NaN
DHW_HeatPumpWaterHeaterPowerTotal,NaN,NaN,NaN
...,...,...,...
HVAC_HeatPumpIndoorUnitPower,NaN,NaN,NaN
HVAC_HeatPumpOutdoorUnitPower,NaN,NaN,NaN
Load_StatusClothesWasher,Load_ClothesWasherPowerWithStandby,Elec_PowerClothesWasher,NaN


# Phase 2
Find constraints of devices:
o	If there are hard constraints for this appliance only
o	Does it have to be on in a continous period of time? How long does it last?
o	How many watts pr second does it use?
o	How much energy does it need from start to finish?


In [137]:
level1_rules_year1 = json_to_dataframe(year=1, level=1)
level1_rules_year1

,pattern,supp,conf,time
0,Load_StatusMBALights,0.991781,1.0,"{'0': [['2013-07-01 03:29:17', '2013-07-01 03:..."
1,Load_StatusSensHeatPrntBDOWN,0.991781,1.0,"{'0': [['2013-07-01 05:58:17', '2013-07-01 06:..."
2,Load_StatusKitchenLights,0.991781,1.0,"{'0': [['2013-07-01 05:58:17', '2013-07-01 08:..."
3,Load_StatusPlugLoadBlender,0.980822,1.0,"{'0': [['2013-07-01 06:03:17', '2013-07-01 06:..."
4,Load_StatusPlugLoadCoffeeMaker,0.980822,1.0,"{'0': [['2013-07-01 06:08:17', '2013-07-01 06:..."
5,Load_StatusPlugLoadMBRTV,0.843836,1.0,"{'0': [['2013-07-01 06:13:17', '2013-07-01 06:..."
6,Load_StatusPlugLoadToasterOven,0.980822,1.0,"{'0': [['2013-07-01 06:13:17', '2013-07-01 06:..."
7,Load_StatusMasterBedroomLights,0.991781,1.0,"{'0': [['2013-07-01 06:13:17', '2013-07-01 06:..."
8,Load_StatusPlugLoadToaster,0.980822,1.0,"{'0': [['2013-07-01 06:18:17', '2013-07-01 06:..."
9,Load_StatusBA2Lights,0.991781,1.0,"{'0': [['2013-07-01 06:28:17', '2013-07-01 06:..."


In [144]:
def find_intervals():
    time_delta_dict = {}
    for year in [1, 2]:
        time_delta_dict[f'year{year}'] = time_delta_dict.get(f'year{year}', {})
        for index, row in json_to_dataframe(year=year, level=1).iterrows():
            if 'SensHeat' not in row['pattern']:
                time_delta_dict[f'year{year}'][row["pattern"]] = pd.Series(dtype='float64')
                for intervals in row["time"].values():
                   for interval in intervals:
                       time_delta_dict[f'year{year}'][row["pattern"]] = pd.concat((time_delta_dict[f'year{year}'][row["pattern"]], pd.Series((pd.to_datetime(interval[1], format="%Y-%m-%d %H:%M:%S") - pd.to_datetime(interval[0], format="%Y-%m-%d %H:%M:%S")).total_seconds()/60)), ignore_index=True)
    return time_delta_dict

In [145]:
time_delta_dict = find_intervals()

In [170]:
def describe_intervals(att):
    if att not in time_delta_dict['year1'].keys():
        return time_delta_dict['year2'][att].describe()
    elif att not in time_delta_dict['year2'].keys():
        return time_delta_dict['year1'][att].describe()
    else:
        return pd.concat((time_delta_dict['year1'][att], time_delta_dict['year2'][att]), axis=0).describe()


apps = set(list(time_delta_dict['year1'].keys()) + list(set(time_delta_dict['year2'].keys())))

for pattern in apps:
    if 'SensHeat' not in pattern:
        print(pattern, describe_intervals(pattern))

Load_StatusDryerPowerTotal count    534.000000
mean      32.749969
std        8.882232
min        1.000000
25%       27.000000
50%       32.000000
75%       38.000000
max       55.000000
dtype: float64
Load_StatusBA2Lights count    2881.000000
mean       16.593168
std         9.892030
min         9.983333
25%        10.000000
50%        15.000000
75%        15.016667
max        60.000000
dtype: float64
Load_StatusPlugLoadToaster count    719.000000
mean      10.999907
std        0.052793
min       10.000000
25%       11.000000
50%       11.000000
75%       11.000000
max       12.000000
dtype: float64
Load_StatusPlugLoadLRBlueRay count    209.000000
mean      89.208692
std        7.346551
min       15.000000
25%       90.000000
50%       90.000000
75%       90.000000
max       90.000000
dtype: float64
Load_StatusPlugLoadHeatingPad count    719.000000
mean      13.572346
std        2.258394
min        9.983333
25%       10.000000
50%       15.000000
75%       15.000000
max       15.01666

In [ ]:
time_delta_dict['year1']['Load_StatusClothesWasher'][lambda self: self <130]

In [150]:
def find_quantile(quantile, att):
    return time_delta_dict['year1'][att].quantile(quantile), time_delta_dict['year2'][att].quantile(quantile)
find_quantile(0.90, "Load_StatusMBALights")

(20.0, 20.0)

In [171]:
hist = pd.Series(time_delta_dict['year1']["Load_StatusApplianceDishwasher"])
fig = px.histogram(hist)
fig.show()

In [ ]:

fig = px.box(time_delta_dict['year1'], log_y=True)
fig.update_traces(jitter= 0.3)
# fig.show()
fig.write_html(Db.get_save_file_directory("rule_constraints/time_delta_boxplot.html"))

# Phase 3
Maybe the patterns can help some of the constraints
    o	Washing machine can be used at every hour, but maybe the constraints of the user can be found in the patterns
        	E.g. General constraint that the appliances cant be started when the users are not home
        	Dryer have to be started after the clotheswasher, so the clotheswasher cannot be started right before they go out of the house.